# Title

## Introduction

Minecraft is a sandbox video game where players explore and build in a world made of blocks. In this world of infinite possibilities, we would like to know the behaviors of the players and their demographics. Data was collected by a Computer Science research group at UBC led by Frank Wood. The method in which they collected data was recording players' actions throughout the world in their Minecraft servers. They provide two data sets, one of player data that contains a list of unique players and their data, and another that contains a list of individual game sessions made by each player. Specifically, we would like to know what Minecraft player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how each player differs from each other.
### Question:
Can the variables played_hours, gender, experience, and age predict whether or not a Minecraft player will subscribe to a game-related newsletter, and how do these predictors vary from player to player?

As we are only looking at player demographic and behavioural data, we will only use the first data set that contains the list of players and their respective game data. This data set contains 196 total observations and 7 variables: subscribe, gender, age, name, played_hours, hashedEmail and experience. Out of all the variables, the predictors of subscribing are experience, played_hours, gender and age. The columns email and name will be left out as these varibles do not contribute to newsletter subscription.

- The subscribe variable is categorical and contains 2 possible values: "TRUE" or "FALSE". This shows whether a player has subscribed to a game related newsletter or not.

- The experience variable is categorical and contains 5 possible values: "Beginner", "Amateur", "Regular", "Pro" and "Veteran". These placeholders describe the level of skill and experience each player has rated themselves from least to most experienced.

- The played_hours variable is numeric and up to 1 decimal place. This variable, quantified in hours, was determined through how long each player played Minecraft in one sitting.

- The gender variable is categorical and contains 7 options: "Male", "Female", "Non-binary", "Prefer not to say", "Agender", "Two-spirited" and "Other." This variable shows the gender of the player.

- The age variable is numeric and contains integer values. This variable is the age of the player.

A noticeable issue is that this data contains some outliers in the played_hours variable. Some players have claimed to play over 200 hours of Minecraft in a single session. These points may affect the classification model in predictions. 



### players_data Information
#### Dataframe
- 196 Observations
- 7 Variables
- Experience was collected from players themselves
- Each name is unique currently but that is not guaranteed
- Issue: Age has 2 N/A values
- Note: Age starts with a capital A
#### Variables
| Name | Type | Description |
| :------- | :------ | :-------
| experience | chr | Categorical (Pro, Veteran, Amateur, Regular, Beginner)  | 
| subscribe | lgl | Categorical (TRUE/FALSE) |
| hashedEmail | chr | Identifies players |
| played_hours | dbl | Numerical (Total amount of hours played) |
| name | chr | Identifies players |
| gender | chr | Categorical (Male, Female, Non-binary, Prefer not to say, Agender, Two-Spirited, Other) |
| Age | dbl | Numerical (Age of the player) |
#### Summary Statisitics
| Name | Unique Observations | Max | Min | Mean |
| :------- | :------ | :-------| :-------| :-------
| played_hours | 43 | 223.1 | 0 | 5.85 |
| Age | 33 | 58 | 9 | 21.14 |
| experience | 5 | - |  - | - | 
| subscribe | 2 | - | - | - | 
| hashedEmail | 196 | - | - | - |  
| name | 196 | - | - | - | 
| gender | 7 | - | - | - |  